In [3]:
import os
import sys

SPARK_HOME = "/usr/hdp/current/spark2-client"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.7-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [4]:
import random
from pyspark import SparkContext, SparkConf

spark_ui_port = random.choice(range(10000, 11000))
print(f"Spark UI port: {spark_ui_port}")

conf = SparkConf()
conf.set("spark.ui.port", spark_ui_port)

sc = SparkContext(appName="BFS", conf=conf)

Spark UI port: 10314


In [22]:
rdd = sc.textFile('/datasets/twitter/twitter_sample_small.tsv')

In [198]:
graph = rdd.map(lambda x : x.split('\t')[::-1]).cache()
links = graph.groupByKey().mapValues(list).cache()
v = links.count()

In [282]:
sourse = '12'
target = '34'


In [283]:
def add_node(node):
    if(node[0]==sourse):
        return (node[0],(node[1],0,[],'entered'))
    return (node[0],(node[1],2*v,[],'not entered'))

In [284]:
matrix = links.map(add_node)

In [285]:
matrix.take(5)

[('2241', (['12'], 107946, [], 'not entered')),
 ('13349', (['12'], 107946, [], 'not entered')),
 ('82473', (['12'], 107946, [], 'not entered')),
 ('414853', (['12', '633'], 107946, [], 'not entered')),
 ('755452', (['12', '456'], 107946, [], 'not entered'))]

In [286]:
queue = matrix.filter(lambda x: x[1][3]=='entered')

In [287]:
def step(node):
    status = node[1][3]
    if(status=='entered'):
        v = node[0]
        neigh = node[1][0]
        dist = node[1][1]
        for u in neigh:
            udist = dist+1
            uparent = v
            ustatus = 'entered'
            entry = (u,([],udist,[uparent],ustatus))
            yield entry
        entry = (node[0],(node[1][0],node[1][1],[],'ready'))
        yield entry
    yield node

In [288]:
def update(version1,version2):
    neigh = version1[0]
    if(len(version2[0])>len(neigh)):
        neigh = version2[0]
    
    dist = min(version1[1],version2[1])
   
    parents = []
    if(version1[1]==dist):
        parents.extend(version1[2])
    if(version2[1]==dist):
        parents.extend(version2[2])
    
    mapping = {'not entered':0,'entered':1,'ready':2}
    inverse_mapping = {0:'not entered',1:'entered',2:'ready'}
    status = inverse_mapping[max(mapping[version1[3]],mapping[version2[3]])]
       
    return (neigh,dist,parents,status)
        

In [289]:
n_iters = 0
while(not queue.isEmpty() and n_iters!=20):
    matrix = matrix.flatMap(step)
    matrix = matrix.reduceByKey(update)
    queue = matrix.filter(lambda x: x[1][3]=='entered')
    n_iters+=1

In [290]:
n_iters

9

In [292]:
matrix.filter(lambda x: x[0]==target).collect()

[('34', ([], 8, ['274'], 'ready'))]